# Importando librerías

In [67]:
import krakenex
import pandas as pd
import json
from datetime import datetime
import ipywidgets as widgets
import plotly.graph_objects as go

# Definiendo API de Kraken

In [68]:
kraken = krakenex.API()

# Definiendo criptomonédas a analizar y tipos de cambio

In [69]:
cryptos = ['BTC', 'ETH']
currencies = ['USD', 'EUR']

# Definiendo funciones

## Función de limpieza de datos

In [70]:
def cleaningData(df):

    time_vars = ['time']
    float_vars = ['open', 'high', 'low', 'close', 'vwap', 'volume']

    for var in time_vars:
        df[var] = pd.to_datetime(df[var], unit='s')

    for var in float_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    return df

## Función para extraer los datos

In [71]:
def getData():
    ohlc = []
    for crypto in cryptos:
        for currency in currencies:
            fresh_data = kraken.query_public(
                'OHLC',
                {'pair': crypto + currency}
            )
            ohlc.append(fresh_data)

    return ohlc

## Función para constriuir el Data Frame

In [72]:
def buildDf(data):
    df = pd.DataFrame()
    for pair in data:
        pair_name = list(pair['result'].keys())[0]
        pair_data = pair['result'][pair_name]
        pair_df = pd.DataFrame.from_records(
            pair_data,
            columns=[
                'time',
                'open',
                'high',
                'low',
                'close',
                'vwap',
                'volume',
                'count'
            ])
        pair_df['pair_name'] = pair_name
        df = pd.concat([df, pair_df], axis=0)

    return df

## Función generar media móvil

In [73]:
def calculateMovingAverage(df):
    
    # Identificando todos los pares disponibles en la extracción
    pairs = df['pair_name'].unique()

    # Calculado la media móvil de 25 sesiones para cada par y almacenando en un data frame independiente
    list_of_dfs = []
    for pair in pairs:
        df_pair = df[df['pair_name'] == pair]
        df_pair['SMA25'] = df_pair['close'].rolling(25).mean()
        list_of_dfs.append(df_pair)

    # Uniendo todos los data frames en un solo data frame resultante
    new_df = pd.DataFrame()
    for x in list_of_dfs:
        new_df = new_df.append(x)
    return new_df


# Extrayendo datos

In [74]:
data = getData()

# Construyendo Data Frame

In [75]:
df = buildDf(data)
df = df.reset_index(drop=True)

# Limpiando datos

In [76]:
df = cleaningData(df)

# Calculado la media móvil

In [77]:
df = calculateMovingAverage(df)

/var/folders/2y/tls0s71d3llbk2fk7vt0mgc80000gn/T/ipykernel_822/4185520521.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2y/tls0s71d3llbk2fk7vt0mgc80000gn/T/ipykernel_822/4185520521.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/2y/tls0s71d3llbk2fk7vt0mgc80000gn/T/ipykernel_822/4185520521.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [78]:
df

,time,open,high,low,close,vwap,volume,count,pair_name,SMA25
0,2022-11-09 00:59:00,18338.00,18340.00,18331.20,18340.00,18338.50,2.701165,29,XXBTZUSD,NaN
1,2022-11-09 01:00:00,18336.90,18340.00,18325.50,18325.50,18334.80,3.178520,20,XXBTZUSD,NaN
2,2022-11-09 01:01:00,18325.50,18339.90,18310.00,18335.50,18327.80,3.887147,41,XXBTZUSD,NaN
3,2022-11-09 01:02:00,18335.30,18335.30,18310.00,18310.00,18317.40,2.041337,16,XXBTZUSD,NaN
4,2022-11-09 01:03:00,18312.80,18325.80,18300.00,18312.80,18310.10,0.889282,23,XXBTZUSD,NaN
...,...,...,...,...,...,...,...,...,...,...
2875,2022-11-09 12:54:00,1221.70,1222.07,1219.43,1219.73,1220.99,21.789107,33,XETHZEUR,1226.5196
2876,2022-11-09 12:55:00,1220.74,1220.75,1218.36,1219.02,1219.20,35.984311,35,XETHZEUR,1226.4800
2877,2022-11-09 12:56:00,1218.36,1219.57,1209.71,1209.71,1211.90,312.508706,88,XETHZEUR,1225.9752
2878,2022-11-09 12:57:00,1210.00,1215.49,1206.03,1214.22,1209.87,122.698311,58,XETHZEUR,1225.7148
